In [3]:
import numpy as np
import time 

In [4]:
class NeuralNetwork():
    def __init__(
        self,
        max_epochs = 100,
        momentum = 0.5,
        learning_rate = 0.0001,
        batch_size = 1,
        neuron_hidden_layer = [3,3,3],
        f_activation = 'sigmoid'
    ):
    
        if (batch_size <= 0):
            raise Exception("batch size must be positive")
        if (learning_rate <= 0):
            raise Exception("learning rate must be positive")
        if (momentum < 0):
            raise Exception("momentum cannot be negative")
        if len(neuron_hidden_layer) == 0 or len(neuron_hidden_layer) == 10:
            raise Exception("learning_rate must be > 0 and <= 10")
    
        self.max_epochs = max_epochs
        self.momentum = momentum
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.neuron_hidden_layer = neuron_hidden_layer
        self.f_activation = f_activation
    
    def sigmoid(self, value):
        return 1 / (1 + np.exp(-value))
    
    def __sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def __activation(self, value , func = 'sigmoid'):
        if func == 'sigmoid':
            return self.sigmoid(value)
        
    def __activation_derivative(self, value , func = 'sigmoid'):
        if func == 'sigmoid':
            return self.__sigmoid_derivative(value)
        
    def __error_derivative(self, pred, actual):
        return (pred - actual) / (pred - np.square(pred))
        
    def __error(self, pred, actual):
        return -(np.log(pred) if actual > 0 else np.log(1.0 - pred))

    def __init_neuron_weight(self, n_feature):
        self.weights = []
        self.bias = []
        prev_input = n_feature
        for i, n_neuron in enumerate(self.neuron_hidden_layer):
            limit = np.sqrt(6 / (prev_input + n_neuron))
            self.weights.append(np.random.rand(n_neuron, prev_input))
            self.bias.append(np.random.rand(n_neuron))
            prev_input = n_neuron
        limit = np.sqrt(6 / (prev_input + n_neuron))
        self.weights.append(np.random.rand(1, prev_input))
        self.weights = np.array(self.weights)
        self.bias.append(np.random.rand(1))
        self.bias = np.array(self.bias)
        
    def get_weights(self):
        return self.weights, self.bias
    
    def predict(self, data):
        result = []
        for x in data:
            prev_input = x
            for i, (w_n, w_b) in enumerate(zip(self.weights, self.bias)):
#                 print("{} | {} | {} | {} | {}".format(i, prev_input, w_n, w_b, np.dot(w_n, prev_input) + w_b))
                out_h_layer = np.dot(w_n, prev_input) + w_b
                prev_input = [self.__activation(out) for out in out_h_layer] 
                
            result.append(self.__activation(prev_input[0]))
        return result
    
    def fit(self, train_x, train_y):
        n_data, n_feature = train_x.shape
        n_batch = int(n_data / self.batch_size)
        batch_size = self.batch_size
        
        self.__init_neuron_weight(n_feature)
        print(self.weights)
        print(self.bias)
        
        for itr in range(self.max_epochs):
            last_delta_weight = [np.zeros(weight.shape) for weight in self.weights]
            last_delta_bias = [np.zeros(bias.shape) for bias in self.bias]
            
            for n in range(0, n_data, batch_size):
                batch_x = train_x[n : n + batch_size]
                batch_y = train_y[n : n + batch_size]
                
                # print("{}".format(batch_x))
                
                delta_weight = [np.zeros(weight.shape) for weight in self.weights]
                delta_bias = [np.zeros(bias.shape) for bias in self.bias]
            
                for x, y in zip(batch_x, batch_y):
                    # feed forward
                    # print(x)
                    prev_input = x
                    output_h_layer = []
                    for i, (w_n, w_b) in enumerate(zip(self.weights, self.bias)):
                        out_h_layer = np.dot(w_n, prev_input) + w_b
                        prev_input = [self.__activation(out) for out in out_h_layer] 
                        output_h_layer.append(np.array(prev_input))
                    
                    # print("out {}".format(output_h_layer))
                    # backprop
                    error_h_layer = [np.zeros(o_l.shape) for o_l in output_h_layer]
                    error_h_layer[-1] = np.array(self.__error_derivative(output_h_layer[-1], y))
                    for i in reversed(range((len(error_h_layer) - 1))):
                        # print("{} | {} | {} | {}".format(i, self.weights[i+1], error_h_layer[i+1].T, "tes"))
                        error_h_layer[i] = (np.dot(error_h_layer[i+1].T, self.weights[i+1]) 
                                            * self.__activation_derivative(output_h_layer[i]))
                        
                    # print("err {}".format(error_h_layer))
                        
                    delta_weight[0] += (self.learning_rate * np.dot(np.array([error_h_layer[0]]).T, 
                                        np.array([x])) + last_delta_weight[0]*self.momentum)
                    delta_bias[0] += self.learning_rate * error_h_layer[0] + last_delta_bias[0] * self.momentum
                    for i in range(1, len(self.weights)):
                        delta_weight[i] += (self.learning_rate * np.dot(np.array([error_h_layer[i]]).T, 
                                            np.array([output_h_layer[i-1]])) + last_delta_weight[i]*self.momentum)
                        delta_bias[i] += self.learning_rate * error_h_layer[i] + last_delta_bias[i] * self.momentum
                        
                    
                # update weight
                delta_weight = np.array(delta_weight) / len(batch_x)
                delta_bias = np.array(delta_bias) / len(batch_x)
                
                self.weights -= delta_weight
                self.bias -= delta_bias
                
                last_delta_weight = delta_weight
                last_delta_bias = delta_bias
                
            # print("{} ======================================== ".format(itr))
        
        return self
                    


In [ ]:
test = NeuralNetwork(max_epochs=100, batch_size=1, neuron_hidden_layer=[2,2])

train_x = np.array([[10,10],[-2,-2]])
train_y = np.array([1,0])

start_time = time.time()
test.fit(train_x, train_y)
print("--- %s seconds ---" % (time.time() - start_time))

print(test.predict(train_x))

test.get_weights()